In [33]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [34]:
import pandas as  pd
import numpy as np
import warnings
import itertools
import xgboost as xgb
from numpy import loadtxt
warnings.filterwarnings('ignore')
kernel_with_output = True

In [35]:
if kernel_with_output:
    df_items=pd.read_csv("../data/items.csv")
    df_categories = pd.read_csv("../data/item_categories.csv")
    df_shops = pd.read_csv("../data/shops.csv")
    df_sales = pd.read_csv("../data/sales_train.csv.gz")
    df_test = pd.read_csv("../data/test.csv.gz")
    temp = pd.read_csv("../data/sample_submission.csv.gz")

In [36]:
df_sales["date"]=pd.to_datetime(df_sales["date"],format="%d.%m.%Y")
df_sales["month"]=df_sales["date"].dt.month
df_sales["year"] = df_sales["date"].dt.year

In [37]:
all_data = pd.read_csv("../results/all_data.csv")

### XGBoost baseline

In [39]:
if kernel_with_output:
    # Extract features and target we want
    baseline_features = ['shop_id', 'item_id', 'item_category_id', 'date_block_num', 'item_cnt_month']
    train = all_data[baseline_features]
    # Remove pandas index column
    train = train.set_index('shop_id')
    train["item_cnt_month"] = train["item_cnt_month"].astype(int)
    train['item_cnt_month'] = train["item_cnt_month"].fillna(0).clip(0,20)
    # Save train set to file
    train.to_csv('../results/train_baseline.csv')


In [40]:
if kernel_with_output:
    dataset = loadtxt('../results/train.csv', delimiter="," ,skiprows=1, dtype = int)
    # trainx's columns: 'shop_id', 'item_id', 'item_category_id', 'date_block_num'
    trainx = dataset[:, 0:4] # without header 
    trainy = dataset[:, 4]

    # Make test_dataset pandas data frame, add category id from df_items, and fix date block num 33
    merged_test = pd.merge(df_test, df_items, on = ['item_id'])[['shop_id','item_id','item_category_id']]
    merged_test['date_block_num'] = 33
    merged_test.set_index('shop_id')
    

,item_id,item_category_id,date_block_num
shop_id,,,
5,5037,19,33
4,5037,19,33
6,5037,19,33
3,5037,19,33
2,5037,19,33
7,5037,19,33
10,5037,19,33
12,5037,19,33
28,5037,19,33


In [42]:
if kernel_with_output:
    model = xgb.XGBRegressor(max_depth = 10, min_child_weight=0.5, subsample = 1, eta = 0.3, num_round = 50, seed = 1)
    model.fit(trainx, trainy, eval_metric='rmse')

[15:25:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:25:26] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, eta=0.3, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=0.5, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, num_round=50, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=1, silent=None, subsample=1, verbosity=1)

In [43]:
 preds = model.predict(merged_test.values)

In [44]:
if kernel_with_output:
    df = pd.DataFrame(preds, columns = ['item_cnt_month'])
    df['ID'] = df.index
    df = df.set_index('ID')
    df.head(10)
    df.to_csv('../results/03.csv')

,item_cnt_month
ID,
0,0.328741
1,0.212095
2,0.350097
3,0.260338
4,0.456667
5,0.423154
6,0.047554
7,0.242267
8,1.137633


#### result 1.35